In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/Advanced_ML_anomaly_detection_L3/DataSets.zip

--2021-11-12 09:55:10--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/Advanced_ML_anomaly_detection_L3/DataSets.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 402533304 (384M) [application/zip]
Saving to: ‘DataSets.zip’

DataSets.zip        100%[===================>] 383.88M  25.4MB/s    in 15s     

2021-11-12 09:55:25 (25.9 MB/s) - ‘DataSets.zip’ saved [402533304/402533304]



In [ ]:
!pip install gensim
!pip install python-Levenshtein
!pip install tensorflow==2.2.0

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ElTree
import re, h5py, itertools, math, glob, zipfile, os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import log_loss, auc, roc_curve
from tensorflow.keras import layers
from tensorflow.keras.layers.core import Masking
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, TimeDistributed
from keras.layers.wrappers import Bidirectional
from tensorflow.keras.models import Model, Sequential, load_model
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from lxml import etree
from gensim.models import Word2Vec

# %matplotlib inline
plt.rcParams['figure.figsize'] = (15, 5)
plt.style.use('ggplot')
seed = 42

import warnings
warnings.filterwarnings(action = "ignore")

ModuleNotFoundError: ignored

In [ ]:

!pip install --upgrade tensorflow-gpu==2.2.0

     |████████████████████████████████| 516.2 MB 4.0 kB/s 
  Attempting uninstall: tensorflow-gpu
    Found existing installation: tensorflow-gpu 2.7.0
    Uninstalling tensorflow-gpu-2.7.0:
      Successfully uninstalled tensorflow-gpu-2.7.0


In [ ]:
pd.set_option("precision", 3)
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'accuracy' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'accuracy' in s and 'val' in s]
    
    plt.figure(figsize = (12, 5), dpi = 100)
    COLOR = 'gray'
    
    plt.rc('legend', fontsize = 14)   
    plt.rc('figure', titlesize = 12)  
        
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
  
    epochs = range(1, len(history.history[loss_list[0]]) + 1)
    
   
    plt.subplot(1, 2, 1)
    plt.subplots_adjust(wspace = 2, hspace = 2)
    plt.rcParams['text.color'] = 'black'
    plt.rcParams['axes.titlecolor'] = 'black'
    plt.rcParams['axes.labelcolor'] = COLOR
    plt.rcParams['xtick.color'] = COLOR
    plt.rcParams['ytick.color'] = COLOR
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b-o',
                 label = 'Train (' + str(str(format(history.history[l][-1],'.4f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g',
                 label = 'Valid (' + str(str(format(history.history[l][-1],'.4f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend(facecolor = 'gray', loc = 'best')
    plt.grid(True)
    plt.tight_layout()
    
   
    plt.subplot(1, 2, 2)
    plt.subplots_adjust(wspace = 2, hspace = 2)
    plt.rcParams['text.color'] = 'black'
    plt.rcParams['axes.titlecolor'] = 'black'
    plt.rcParams['axes.labelcolor'] = COLOR
    plt.rcParams['xtick.color'] = COLOR
    plt.rcParams['ytick.color'] = COLOR
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b-o',
                 label = 'Train (' + str(format(history.history[l][-1],'.4f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g',
                 label = 'Valid (' + str(format(history.history[l][-1],'.4f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend(facecolor = 'gray', loc = 'best')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

class B_Generator(object):
    def __init__(self, BZ, XX, YY, ohe):
        self.BZ = BZ
        self.n_b = int(math.floor(np.shape(XX)[0] / BZ))
        self.b_index = [a * BZ for a in range(0, self.n_b)]
        self.XX = XX
        self.YY = YY
        self.ohe = ohe
        
    def __iter__(self):
        for var_0 in itertools.cycle(self.b_index):
            YY = self.YY[var_0 : (var_0 + self.BZ)]
            ohe_Y = self.ohe.transform(YY.reshape(len(YY), 1))
            yield (self.XX[var_0 : (var_0 + self.BZ),], ohe_Y)

In [ ]:
number = 2

if zipfile.is_zipfile('DataSets.zip'):
  file_1 = zipfile.ZipFile('DataSets.zip', 'r')
else:
  print('Type file isn`t ZIP')

name_dataset = file_1.namelist()[number]
file_1.extract(name_dataset)
print("File", name_dataset, "has been read")

File TestbedSunJun13Flows.xml has been read


In [ ]:
tree_set = ElTree.parse(name_dataset)
root_tree_set = tree_set.getroot()

result = []
var_1 = root_tree_set.items()[0][1][:-4]

for item in root_tree_set.findall(var_1):
    result.append({node.tag: node.text for node in item.getiterator()})

dSET = pd.DataFrame(result)
dSET = dSET.drop(dSET.columns[[0]], axis = 1)
dSET = dSET.drop_duplicates()

dSET = dSET.sort_values('startDateTime')
dSET['IPs_sequence'] = dSET['source'] + '_' + dSET['destination'] + '_' + dSET['startDateTime'].str[:13]

dSET['res_port'] = np.where(dSET.destinationPort <= dSET.sourcePort,
                            dSET['destinationPort'],
                            dSET['sourcePort'])


dSET = dSET.rename(columns = {'totalSourceBytes': 'totSB',
                        'totalDestinationBytes': 'totDB',
                        'totalDestinationPackets': 'totDP',
                        'totalSourcePackets': 'totSP',
                        'sourcePayloadAsBase64': 'sourB64',
                        'sourcePayloadAsUTF': 'sourUTF',
                        'destinationPayloadAsBase64': 'destB64',
                        'destinationPayloadAsUTF': 'destUTF',
                        'direction': 'direct',
                        'sourceTCPFlagsDescription': 'sourTCPFd',
                        'destinationTCPFlagsDescription': 'destTCPFd',
                        'protocolName': 'pName',
                        'sourcePort': 'sPort',
                        'destination': 'dest',
                        'destinationPort': 'dPort'})
print("Preparation process has been finished")

Preparation process has been finished


In [ ]:
dSET.shape, dSET.columns

((137160, 22),
 Index(['appName', 'totSB', 'totDB', 'totDP', 'totSP', 'sourB64', 'sourUTF',
        'destB64', 'destUTF', 'direct', 'sourTCPFd', 'destTCPFd', 'source',
        'pName', 'sPort', 'dest', 'dPort', 'startDateTime', 'stopDateTime',
        'Tag', 'IPs_sequence', 'res_port'],
       dtype='object'))

In [ ]:
dSET.head(5)

,appName,totSB,totDB,totDP,totSP,sourB64,sourUTF,destB64,destUTF,direct,sourTCPFd,destTCPFd,source,pName,sPort,dest,dPort,startDateTime,stopDateTime,Tag,IPs_sequence,res_port
0,Unknown_UDP,2633658,0,0,28971,None,None,None,None,L2R,N/A,N/A,192.168.5.122,udp_ip,5353,224.0.0.251,5353,2010-06-12T23:57:24,2010-06-13T09:24:52,Normal,192.168.5.122_224.0.0.251_2010-06-12T23,5353
2,HTTPWeb,64,128,2,1,None,None,None,None,L2L,"F,A",R,192.168.2.113,tcp_ip,4191,192.168.5.122,80,2010-06-12T23:57:38,2010-06-12T23:59:20,Normal,192.168.2.113_192.168.5.122_2010-06-12T23,4191
4,HTTPWeb,128,64,1,2,None,None,None,None,L2R,"F,A","F,A",192.168.2.113,tcp_ip,4192,207.241.148.80,80,2010-06-12T23:57:40,2010-06-12T23:59:20,Normal,192.168.2.113_207.241.148.80_2010-06-12T23,4192
6,HTTPImageTransfer,128,128,2,2,None,None,None,None,L2R,"F,A","F,A",192.168.2.110,tcp_ip,1864,216.49.88.12,80,2010-06-12T23:57:42,2010-06-12T23:59:22,Normal,192.168.2.110_216.49.88.12_2010-06-12T23,1864
8,SecureWeb,64,0,0,1,None,None,None,None,L2R,"F,A",None,192.168.2.113,tcp_ip,4186,63.245.209.72,443,2010-06-12T23:58:11,2010-06-12T23:59:07,Normal,192.168.2.113_63.245.209.72_2010-06-12T23,4186


In [ ]:

print("Stage I. Keys building\n")
key = dSET.groupby('IPs_sequence')[['Tag', 'res_port']].agg({"Tag": lambda var_2: "%s" % ','.join([var_3 for var_3 in var_2]),
          "res_port" :lambda var_2: "%s" % ','.join([str(var_3) if int(var_3) < 10000 else "10000" for var_3 in var_2])})

print("Unique keys:\n" + str(key.count()))
attacks = [var_4.split(",") for var_4 in key.Tag.tolist()]
sequences = [var_4.split(",") for var_4 in key.res_port.tolist()]

Stage I. Keys building

Unique keys:
Tag         11243
res_port    11243
dtype: int64


In [ ]:
print("Stage II. Label encoding\n")
U_tokens = list(set([var_5 for var_6 in sequences for var_5 in var_6]))
print("Number of unique tokens :", len(U_tokens))
LE = LabelEncoder().fit(U_tokens)
sequences = [LE.transform(var_7).tolist() for var_7 in sequences]
sequences = [[var_6 + 1 for var_6 in var_5] for var_5 in sequences]
print("Number of sequences :", len(sequences))
sequence_attack = zip(attacks, sequences)

Stage II. Label encoding

Number of unique tokens : 4168
Number of sequences : 11243


In [ ]:
print("Stage III. Sequences generating for the future model\n")
var_8 = np.float32(0)
len_sequence = 10
print("Length of the primary sequence :", len_sequence)
seq_IDX, seq_X, seq_Y, seq_ATT = [], [], [], []
for var_10, (var_11, var_12) in enumerate(sequence_attack):
    sequence_1 = [np.float32(0)] * (len_sequence) + var_12
    sequence_2 = [np.float32(0)] * (len_sequence) + var_11
    for var_9 in range(len_sequence, len(sequence_1)):
        sequence_3 = sequence_1[(var_9 - len_sequence):(var_9)]
        var_14 = []
        for var_13 in sequence_3:
            try:
                var_14.append(var_13)
            except:
                var_14.append(var_8)
        seq_X.append(var_14)
        seq_Y.append(sequence_1[var_9])
        seq_IDX.append(var_10)
        seq_ATT.append(sequence_2[var_9])
print("Length of X & Y sets :", len(seq_X))

Stage III. Sequences generating for the future model

Length of the primary sequence : 10
Length of X & Y sets : 137160


In [ ]:
print("Stage IV. One-hot-encoder initializing\n")
OHE = OneHotEncoder(sparse = False, categories = 'auto').fit(np.unique(seq_Y).reshape(-1, 1))

X = np.array(seq_X)
print("Dimensionality size of set X :", X.shape)

Stage IV. One-hot-encoder initializing

Dimensionality size of set X : (137160, 10)


In [ ]:
from tensorflow import keras
from keras.models import Sequential
print("Stage V. Model building\n")
drop_level = 0.35
N_neurons = 50 

model = Sequential()
model.add(layers.Embedding(output_dim = 100,
                    input_dim = len(U_tokens) + 1,
                    mask_zero = True))

model.add(layers.Bidirectional(LSTM(N_neurons, return_sequences = True)))
model.add(layers.Dropout(drop_level))


model.add(layers.Bidirectional(LSTM(N_neurons, activation = "relu", return_sequences = False)))
model.add(layers.Dropout(drop_level))

model.add(layers.Dense(N_neurons, activation = "linear"))
model.add(layers.Dropout(drop_level))

model.add(layers.Dense(len(U_tokens), activation = "softmax"))

model.summary()

ImportError: ignored

In [7]:
print("Stage VI. Compile and fit the model\n")

batch_size = 512 
n_epochs = 10     

optim = tf.keras.optimizers.Nadam()   
loss_f = tf.keras.metrics.categorical_crossentropy

T_data = B_Generator(batch_size, np.asarray(X), np.asarray(seq_Y), OHE)

model.compile(loss = loss_f,
              optimizer = optim,
              metrics = ["accuracy"])

history = model.fit_generator(T_data.__iter__(),
    steps_per_epoch = T_data.n_b,
    epochs = n_epochs,
    verbose = 1)

Stage VI. Compile and fit the model



NameError: ignored

In [ ]:
print("Stage VII. Results visualization\n")
plot_history(history)

In [ ]:
print("Stage VIII. Model saving & prediction checking\n")

M_name = "Model"

filepath = M_name + '.h5'
tf.keras.models.save_model(model, filepath, include_optimizer = True, save_format = 'h5', overwrite = True)
print("Size of the saved model :", os.stat(filepath).st_size, "bytes")

In [ ]:
model_L = tf.keras.models.load_model(filepath)
predicts = model_L.predict(X, batch_size = batch_size)
print("Dimensionality sizes of model predicts :", predicts.shape, "\n")
print("Compare with length of X & Y sets :\t", len(seq_X), "\nand with number of tokens :\t\t", len(U_tokens))